In [1]:
import pandas as pd
import talib
from enum import Enum
import QuantLib as ql
from market import MarketData
from datetime import date, timedelta
from pricing import *


In [ ]:

def get_dates_between(start_date, end_date):
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)
    return dates

class Backtester(ABC):
    def __init__(self, universe, start_date, end_date, starting_balance):
        self.starting_balance = starting_balance
        self.calendar = ql.TARGET()

        market = MarketData(start_date, end_date)
        self.portfolio = Portfolio()
        for symbol in universe:
            market.build(symbol)
        self.available_market_data = market.market_data

        self.backtest_dates = get_dates_between(start_date, end_date)
        self.history = {}
        self.ref_dates = []
        self.unprocessed_dates = []
        self.positions_summary = None

    def backtest(self):
        """ chiama in automatico on_data a ogni iterazione e aggiorna history con i valori noti fino alla data"""
        self.positions_summary = pd.DataFrame()
        self.ref_dates = []
        for backtest_date in self.backtest_dates:
            if self.calendar.isHoliday(ql.Date.from_date(backtest_date)):
                continue
            try: 
                self.history[backtest_date] = self.available_market_data[backtest_date]
                # aggiungere if is_valid anche come controllo da far fare in fase di preprocessing
            except:
                self.unprocessed_dates.append(backtest_date)
                continue
            self.ref_dates.append(backtest_date)
            self.on_data()
            summary = pd.DataFrame(self.portfolio.get_positions_summary(self.available_market_data[backtest_date]))
            self.positions_summary = pd.concat([self.summary_df, summary], axis = 0)
            # funzione che aggiorna il portafoglio in automatico

    def get_data_at(self, ref_date):
        history = {}
        for backtest_date in self.backtest_dates:
            if backtest_date > ref_date:
                break
            if self.calendar.isHoliday(ql.Date.from_date(backtest_date)):
                continue
            try: 
                history[backtest_date] = self.available_market_data[backtest_date]
                # aggiungere if is_valid anche come controllo da far fare in fase di preprocessing
            except:
                continue
        return history
                
    @abstractmethod
    def on_data(self):
        """
        Implement trading logic.
        """
        pass


class BacktesterConcrete(Backtester):
    def __init__(self, universe, start_date, end_date, starting_balance):
        super().__init__(universe, start_date, end_date, starting_balance)

    def on_data(self):
        ref_date = self.ref_dates[-1] 
        # print("*")


In [42]:
universe = ['ctp1']
start_date = date(2022, 10, 14)
end_date = date(2023, 10, 12)
market = MarketData(start_date, end_date)
calendar = ql.TARGET()
backtester = BacktesterConcrete(universe, start_date, end_date, 100)

In [43]:
backtester.backtest()

In [44]:
backtester.summary_df

""


In [14]:
backtester.get_data_at(date(2022, 10, 18))

{datetime.date(2022, 10, 14): {'ref_date': datetime.date(2022, 10, 14),
  'equity': {'ctp1': {'price': 1.7124, 'div_yield': 0.05, 'is_valid': True}},
  'rate': {'riskfree': 0.03},
  'volatility': {'ctp1': <QuantLib.QuantLib.BlackVarianceSurface; proxy of <Swig Object of type 'ext::shared_ptr< BlackVarianceSurface > *' at 0x000002A95249CB70> >}},
 datetime.date(2022, 10, 17): {'ref_date': datetime.date(2022, 10, 17),
  'equity': {'ctp1': {'price': 1.743, 'div_yield': 0.05, 'is_valid': True}},
  'rate': {'riskfree': 0.03},
  'volatility': {'ctp1': <QuantLib.QuantLib.BlackVarianceSurface; proxy of <Swig Object of type 'ext::shared_ptr< BlackVarianceSurface > *' at 0x000002A95249CBD0> >}},
 datetime.date(2022, 10, 18): {'ref_date': datetime.date(2022, 10, 18),
  'equity': {'ctp1': {'price': 1.8028, 'div_yield': 0.05, 'is_valid': True}},
  'rate': {'riskfree': 0.03},
  'volatility': {'ctp1': <QuantLib.QuantLib.BlackVarianceSurface; proxy of <Swig Object of type 'ext::shared_ptr< BlackVarian

In [15]:
backtester.history

{}